In [98]:
from glob import glob
import urllib
import earthaccess
from datatree import open_datatree
from PIL import Image
import xarray as xr
from satpy.scene import Scene
import os
import re
from pyresample import create_area_def
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from satpy.writers import get_enhanced_image
from math import isnan

In [114]:
def download_files(level, bounding_box, temporal_limits):
    '''Download data from EarthData (NOT cloud access)'''
    
    earthaccess.login()
    #password is OHW2023ohw2023!
    
    if level == 'L1b':
        short_names = ["VNP02MOD", "VNP02IMG", "VNP02DNB", "VNP03MOD", "VNP03IMG", "VNP03DNB",
                      "VJ02MOD", "VJ02IMG", "VJ02DNB", "VJ03MOD", "VJ03IMG", "VJ03DNB"]
    elif level == 'L2':
        short_names = ["VNP29"]
    
    for short_name in short_names:
        results = earthaccess.search_data(
        short_name=short_name, 
        cloud_hosted=True, 
        bounding_box=bounding_box,
        temporal=temporal_limits
        )
        
        earthaccess.download(results, './data/' + level + '/') #Separate directories for each product
        #earthaccess.download(results, './data/' + level + '/' + short_names + '/') #Separate directories for each product

In [2]:
def setup_projection(bounding_box):
    '''Set up a custom area and projection for the images'''
    
    #See also the example on swaths of polar orbiter data in the Satpy half-day tutorial notebook 04_resampling
    
    # Bounding box coordinates
    area_id = 'skq2022'
    description = 'Beaufort Sea 2022 Stereographic Projection'

    area_extent = (bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3])

    width = 1100  # Specify the desired width in pixels
    height = 550  # Specify the desired height in pixels

    projection = {
        'proj': 'stere',
        'lat_0': 90.0,       # Latitude of the projection's center
        'lon_0': -145.0,     # Longitude of the projection's center
        'lat_ts': 60.0,      # Latitude where true scale is used
        'a': 6378273,        # Semi-major axis of the ellipsoid (WGS84)
        'b': 6356889.449,    # Semi-minor axis of the ellipsoid (WGS84)
    }

    #create_area_def?
    my_area = create_area_def(area_id, description = description, projection = projection,
                              width=width, height=height,
                              area_extent=area_extent, units='degrees')

    print(my_area)

    lons, lats = my_area.get_lonlats()

    print("Longitude Range:", lons.min(), lons.max()) 
    print("Latitude Range:", lats.min(), lats.max())

    print("Pixel size: x = ", my_area.pixel_size_x, "m, y = ", my_area.pixel_size_y, "m") #horizontal resolution in meters
    
    return(my_area)


In [4]:
def get_file_times(filenames):
    '''Generate a list of unique, sorted day and time strings for all files.'''
    
    pattern = r'\.A(\d{7}).*(\d{4})\.001\.'  # Capture both day and time components of the filename
    time_strings = [None] * len(filenames)  # Preallocate with None
    for i, filename in enumerate(filenames):
        match = re.search(pattern, filename)
        if match: 
            day = match.group(1)
            hour = match.group(2)
            time_strings[i] = day + '.' + hour
    
    unique_sorted_time_strings = sorted(set(time_strings))
    return(unique_sorted_time_strings) 

In [5]:
def get_filenames_at_current_time(i, filetimes):
    
    '''Generate a list of all filenames at the current timestep. If two sets of files
        are taken six minutes apart, assume they are part of the same swath and group them together.'''
    
    filetime = filetimes[i]
    filenames = glob(data_path+'L1b/VNP*'+filetime+'.001*nc') #Files at this timestep

    #Check if the next batch of files is within six minutes and should be combined
    if i < len(filetimes)-1:
        nextfiletime = filetimes[i+1]
        
        # Convert timestamp strings to datetime objects
        filetime_dt = datetime.strptime(filetime, "%Y%j.%H%M")
        nextfiletime_dt = datetime.strptime(nextfiletime, "%Y%j.%H%M")

        # Calculate time difference
        time_difference = nextfiletime_dt - filetime_dt
        combine_files = time_difference <= timedelta(minutes=6)
        
        if combine_files: 
            print("Combining files at " + filetime[-4:] + ' and ' + nextfiletime[-4:])
            nextfilenames = glob(data_path+'L1b/VNP*'+nextfiletime+'.001*nc')  
            filenames.extend(nextfilenames)
    
    #Check if this batch of files was already combined with the previous batch of files
    if i > 0:
        prevfiletime = filetimes[i-1]
        prevfiletime_dt = datetime.strptime(prevfiletime, "%Y%j.%H%M")

        # Calculate time difference
        time_difference = filetime_dt - prevfiletime_dt
        already_combined = time_difference <= timedelta(minutes=6)
        if already_combined:
            print("Files at " + filetime[-4:] + ' were aleady combined with ' + prevfiletime[-4:]+ '. Will not reprocess the files')
            filenames = None
    
    #print(filenames)
    return filenames

In [111]:
def contains_night_pixels(scn, my_area):
    '''Determine if the resampled scene region contains any nighttime pixels''' 
    solar_zenith_angles = ['solar_zenith_angles', 'dnb_solar_zenith_angle']
    available = scn.available_dataset_names()
    available_zenith_angles = list(filter(lambda item: item in available, solar_zenith_angles))
    
    # Define a threshold solar zenith angle to distinguish day and night
    max_zenith_angle_threshold  = 88
    
    scn.load([available_zenith_angles[0]], generate=False)
    resampled_scene = scn.resample(my_area)
    #min_zenith_angle = np.nanmin(resampled_scene[available_zenith_angles[0]].values)
    
    #From experimenting, it seems like the max angle is important in this case. 
    max_zenith_angle = np.nanmax(resampled_scene[available_zenith_angles[0]].values)
       
    #print('Minumum solar angle in resampled scene: ', min_zenith_angle)
    print('Maximum solar angle in resampled scene: ', max_zenith_angle)

    if max_zenith_angle >= max_zenith_angle_threshold or isnan(max_zenith_angle): #min_zenith_angle <= (90-threshold_angle) or max_zenith_angle >= (90+threshold_angle) or (isnan(min_zenith_angle) and isnan(max_zenith_angle)):
        print('Resampled scene contains some night time pixels')
        return True
    else: 
        print('Resampled scene contains only daytime pixels')
        return False
   

In [11]:
def scn_to_numpy(resampled_scene, channel):
    # Get the image data and lat/lon arrays
    image_data = resampled_scene[channel].values
    longitudes, latitudes = resampled_scene[channel].attrs['area'].get_lonlats()

    # Create a NumPy array by combining image data with lat/lon information
    combined_array = np.dstack((image_data, latitudes, longitudes))

    # Check that data has been exported sensibly
    plot_combined_data(combined_array)


In [115]:
#Set up time and geographic range, download files

bounding_box=(-160.0, 73.0, -130.0, 77.0)
temporal_limits=("2022-10-07", "2022-10-10")
download_files(level='L1b', bounding_box=bounding_box, temporal_limits=temporal_limits)
#download_files(level='L2', bounding_box=bounding_box, temporal_limits=temporal_limits)


EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
You're now authenticated with NASA Earthdata Login
Using token with expiration date: 10/15/2023
Using .netrc file for EDL
Granules found: 80
 Getting 80 granules, approx download size: 10.15 GB


QUEUEING TASKS | : 100%|██████████████████████| 80/80 [00:00<00:00, 6036.82it/s]


File VNP02MOD.A2022280.0124.001.2022280084328.nc already downloaded
File VNP02MOD.A2022280.0130.001.2022280084330.nc already downloaded
File VNP02MOD.A2022280.1000.001.2022280180432.nc already downloaded
File VNP02MOD.A2022280.0124.002.2022280095811.nc already downloaded
File VNP02MOD.A2022280.0130.002.2022280095910.nc already downloaded
File VNP02MOD.A2022280.1000.002.2022280193057.nc already downloaded
File VNP02MOD.A2022280.1006.001.2022280180434.nc already downloaded
File VNP02MOD.A2022280.1006.002.2022280191323.nc already downloaded
File VNP02MOD.A2022280.1142.001.2022280180434.nc already downloaded
File VNP02MOD.A2022280.1506.001.2022280205856.nc already downloaded
File VNP02MOD.A2022280.1506.002.2022280221136.nc already downloaded
File VNP02MOD.A2022280.1142.002.2022280191449.nc already downloaded
File VNP02MOD.A2022280.1324.002.2022280204344.nc already downloaded
File VNP02MOD.A2022280.1324.001.2022280193159.nc already downloaded
File VNP02MOD.A2022280.1648.001.2022280224702.nc

PROCESSING TASKS | : 100%|██████████████████| 80/80 [00:00<00:00, 137743.97it/s]
COLLECTING RESULTS | : 100%|████████████████| 80/80 [00:00<00:00, 328000.31it/s]


Granules found: 80
 Getting 80 granules, approx download size: 13.07 GB


...

Granules found: 82
 Getting 82 granules, approx download size: 5.04 GB


QUEUEING TASKS | : 100%|█████████████████████| 82/82 [00:00<00:00, 15521.84it/s]


File VNP03DNB.A2022280.0124.001.2022280080859.nc already downloadedFile VNP03DNB.A2022280.0130.001.2022280080901.nc already downloaded

File VNP03DNB.A2022280.0130.002.2022280092458.nc already downloaded
File VNP03DNB.A2022280.0818.001.2022280162323.nc already downloaded
File VNP03DNB.A2022280.0124.002.2022280092500.nc already downloaded
File VNP03DNB.A2022280.1000.001.2022280174704.nc already downloaded
File VNP03DNB.A2022280.1000.002.2022280185557.nc already downloaded
File VNP03DNB.A2022280.1142.001.2022280174700.nc already downloaded
File VNP03DNB.A2022280.1142.002.2022280185554.nc already downloaded
File VNP03DNB.A2022280.1006.002.2022280185555.nc already downloaded
File VNP03DNB.A2022280.1506.001.2022280204338.nc already downloaded
File VNP03DNB.A2022280.1506.002.2022280215156.nc already downloaded
File VNP03DNB.A2022280.1324.001.2022280191357.nc already downloaded
File VNP03DNB.A2022280.1324.002.2022280202002.nc already downloaded
File VNP03DNB.A2022280.1006.001.2022280174658.nc

PROCESSING TASKS | : 100%|██████████████████| 82/82 [00:00<00:00, 200543.98it/s]
COLLECTING RESULTS | : 100%|████████████████| 82/82 [00:00<00:00, 138738.58it/s]


Granules found: 0
List of URLs or DataGranule isntances expected
Granules found: 0
List of URLs or DataGranule isntances expected
Granules found: 0
List of URLs or DataGranule isntances expected
Granules found: 0
List of URLs or DataGranule isntances expected
Granules found: 0
List of URLs or DataGranule isntances expected
Granules found: 0
List of URLs or DataGranule isntances expected


In [8]:
#Create a list of times at which images were taken. These are directly from the file names
#The format is YYYYDDD.HHMM

data_path = '/Users/lcrews/Documents/Python/sea_ice_oscillations/data/'
all_filenames = glob(data_path+'L1b/VNP*.001*nc')
filetimes = get_file_times(all_filenames)
print(filetimes)

['2022280.0124', '2022280.0130', '2022280.0818', '2022280.1000', '2022280.1006', '2022280.1142', '2022280.1324', '2022280.1506', '2022280.1642', '2022280.1648', '2022280.1824', '2022280.2006', '2022280.2142', '2022280.2148', '2022280.2324', '2022280.2330', '2022281.0106', '2022281.0942', '2022281.1124', '2022281.1306', '2022281.1448', '2022281.1624', '2022281.1630', '2022281.1806', '2022281.1942', '2022281.1948', '2022281.2124', '2022281.2130', '2022281.2306', '2022282.0048', '2022282.0924', '2022282.1106', '2022282.1248', '2022282.1424', '2022282.1430', '2022282.1606', '2022282.1748', '2022282.1924', '2022282.1930', '2022282.2106', '2022282.2248']


In [113]:
#Create reprojected images at each timestep

my_area = setup_projection(bounding_box) #Setup projection      

#Make  images for each satellite pass
for i, filetime in enumerate(filetimes):
    #if i > 1: continue

    print() # This will print a blank line to distinguish this time's information
    
    #All files at the current time - combine files on same swath (six minutes apart)
    filenames = get_filenames_at_current_time(i, filetimes)
    if filenames is None: #This happens if the timestep was part of the same swath and already plotted at previous timestep
        continue  
    
    scn = Scene(filenames=filenames, reader='viirs_l1b')
    
    # Decide what datasets / composites to plot
    # More info about built-in VIIRS composites here: https://satpy.readthedocs.io/en/stable/api/satpy.composites.viirs.html
    # Even more about the snow_age RGB snow composite at http://dx.doi.org/10.1016/j.rse.2017.04.028
    # During day, consider trying I02 based on https://twitter.com/bill_line/status/1690022728441716736
    if contains_night_pixels(scn, my_area):
        channels = ['adaptive_dnb']
    else:
        channels = ['true_color', 'snow_age']
        
    #Remove any datasets composites that are not available for this scene
    available = scn.available_composite_names() + scn.available_dataset_names()
    channels = list(filter(lambda item: item in available, channels))
        
    for channel in channels:
         
        #Generate composite and resample to the common grid my_area defined in setup_projection
        try:
            
            #Data will have to be resampled before the composite can actually be generated. 
            #This is because different required bands have different resolution. 
            #generate=False allows us to "load" the ccomposite and then resample
            # See also https://satpy.readthedocs.io/en/latest/faq.html#how-can-i-speed-up-creation-of-composites-that-need-resampling
            scn.load([channel], generate=False)
            resampled_scene = scn.resample(my_area)
            #print(np.nanmean(resampled_scene[channel].values))
        
            print('Generating ' + channel + ' at ' + filetime)
            savename = './images/VNP_%s_%s.png' % (filetime, channel)
            resampled_scene.save_dataset(channel, filename = savename)
                             
            #This is done automatically when saving to a .png, but we need to do it before converting to numpy arrays
            #See Satpy tutorial 05_composites: "Earlier in this tutorial we talked about "enhancing" data; 
            #the process of scaling data from observed scientific data values to image values (ex. 0-255 integers). 
            #This process was automatically done for us by the save_datasets. If we want to display our data 
            #in a matplotlib figure we need to do this manually. Satpy provides a get_enhanced_image utility function to simplify this.
            #enhanced_resampled_scene = get_enhanced_image(resampled_scene[channel])
            
            savename = './images/VNP_%s_%s.png' % (filetime, channel)
            if os.path.exists(savename): #Dont remake this figure
                continue
                #Write to an image .png
            resampled_scene.save_dataset(channel, filename = savename)
            
            #Create numpy arrays to use for image analysis
            #scn_to_numpy(enhanced_resampled_scene, channel)
        
        except Exception as e:
            print(f"An error occurred while processing '{channel}' for '{filetime}': {e}")
    

Area ID: skq2022
Description: Beaufort Sea 2022 Stereographic Projection
Projection: {'a': '6378273', 'lat_0': '90', 'lat_ts': '60', 'lon_0': '-145', 'no_defs': 'None', 'proj': 'stere', 'rf': '298.279411123064', 'type': 'crs', 'units': 'm', 'x_0': '0', 'y_0': '0'}
Number of columns: 1100
Number of rows: 550
Area extent: (-461819.8016, -1723534.9635, 352116.2306, -1314115.6629)
Longitude Range: -164.34357984307965 -130.01910355902044
Latitude Range: 73.0042764882928 77.43597501958925
Pixel size: x =  739.9418474860364 m, y =  744.3987284632565 m

Combining files at 0124 and 0130
Maximum solar angle in resampled scene:  87.18
Resampled scene contains only daytime pixels
Generating true_color at 2022280.0124


/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


Generating snow_age at 2022280.0124


/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/enhancements/__init__.py:293: RuntimeWarning: invalid value encountered in cast
  band_data = band_data.clip(0, lut.size - 1).astype(np.uint8)



Files at 0130 were aleady combined with 0124. Will not reprocess the files



...

/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)



Maximum solar angle in resampled scene:  112.74
Resampled scene contains some night time pixels
Generating adaptive_dnb at 2022282.0924


/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/dask/array/chunk.py:278: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)



Maximum solar angle in resampled scene:  113.04
Resampled scene contains some night time pixels
Generating adaptive_dnb at 2022282.1106

Maximum solar angle in resampled scene:  109.95
Resampled scene contains some night time pixels
Generating adaptive_dnb at 2022282.1248

Combining files at 1424 and 1430
Maximum solar angle in resampled scene:  104.29
Resampled scene contains some night time pixels
Generating adaptive_dnb at 2022282.1424

Files at 1430 were aleady combined with 1424. Will not reprocess the files

Maximum solar angle in resampled scene:  98.159996
Resampled scene contains some night time pixels
Generating adaptive_dnb at 2022282.1606

Maximum solar angle in resampled scene:  92.5
Resampled scene contains some night time pixels
Generating adaptive_dnb at 2022282.1748

Combining files at 1924 and 1930
Maximum solar angle in resampled scene:  87.57
Resampled scene contains only daytime pixels
Generating true_color at 2022282.1924
Generating snow_age at 2022282.1924

File

UnboundLocalError: cannot access local variable 'filetime_dt' where it is not associated with a value

In [13]:
def plot_combined_data(combined_array):
    # Assuming combined_array contains image data, latitudes, and longitudes
    image_data = combined_array[:, :, 0]  # Extract image data
    latitudes = combined_array[:, :, 1]  # Extract latitudes
    longitudes = combined_array[:, :, 2]  # Extract longitudes

    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(3, 1, figsize=(5, 15))

    # Plot longitudes
    axs[0].imshow(longitudes, cmap='viridis', origin='upper')
    axs[0].set_title('Longitude Plot')
    axs[0].set_xlabel('Pixel Column')
    axs[0].set_ylabel('Pixel Row')
    axs[0].set_aspect('equal')  # Equal aspect ratio

    # Plot latitudes
    axs[1].imshow(latitudes, cmap='viridis', origin='upper')
    axs[1].set_title('Latitude Plot')
    axs[1].set_xlabel('Pixel Column')
    axs[1].set_ylabel('Pixel Row')
    axs[1].set_aspect('equal')  # Equal aspect ratio

    # Plot image data
    axs[2].imshow(image_data, cmap='gray', origin='upper')
    axs[2].set_title('Image Data Plot')
    axs[2].set_xlabel('Pixel Column')
    axs[2].set_ylabel('Pixel Row')
    axs[2].set_aspect('equal')  # Equal aspect ratio

    # Adjust spacing between subplots
    plt.tight_layout()

    # Show the plots
    plt.show()


In [260]:
from satpy import available_writers
sorted(available_writers())

Could not import writer config from: ['/Users/lcrews/miniconda3/envs/sea-ice-oscillations/lib/python3.11/site-packages/satpy/etc/writers/ninjotiff.yaml']


['awips_tiled', 'cf', 'geotiff', 'mitiff', 'ninjogeotiff', 'simple_image']

In [ ]:
#This has proved unhelpful because it operates on the un-resampled scene so 
#it doesn't reflect what's happening in the sub-region that we're interested in

def get_day_night(filenames):
    all_flags = [None] * len(filenames)
    for i, file in enumerate(filenames):
        dt = open_datatree(file)
        all_flags[i] = dt.attrs['DayNightFlag']
        
    # Check if all strings are the same
    all_same = all(flag == all_flags[0] for flag in all_flags)
    if all_same:
        return(all_flags[0])
    
    else: #This can happen when combining two sets of files taken six minutes apart 
        #print('Unclear when image is taken. Files are:', all_flags)
        
        if 'Day' in all_flags and 'Night' not in all_flags: #Some combo of 'Day' and 'Both'
            return('Day')
        elif 'Night' in all_flags and 'Day' not in all_flags: #Some combo of 'Night' and 'Both'
            return('Night')
        elif 'Day' in all_flags and 'Night' in all_flags: #Some combo of 'Day' and 'Night' - will make both images
            return('Both')
        else:
            return(None)  # No valid combination found
        
def choose_composite_daynightflag(filenames):
    '''Decide which composite to generate for this batch of files''' 
    day_night_flag = get_day_night(filenames)
    print('Image is ', day_night_flag)

    if day_night_flag == 'Day':
        composite = 'true_color'
    elif day_night_flag == 'Night' or day_night_flag == 'Both':
        composite = 'adaptive_dnb'
    elif day_night_flag == 'Both':
        print('Not making image because day_night_flag is Both')
        composite = None
        #Need to decide what to do...maybe make both composites?
    return composite